In [1]:
print("Hello world")

Hello world


In [23]:
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

In [17]:
import importlib
import model  # Ensure model.py is imported
importlib.reload(model)  # Reload the module


<module 'model' from '/home/hiran/Desktop/DL/model.py'>

In [1]:
from keras.datasets import fashion_mnist
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

import importlib
import model  # Ensure model.py is imported
importlib.reload(model)  # Reload the module

X_train = X_train/255.0
X_test = X_test/255.0
import numpy as np
from model import Model  # Import the class

# Create an object of the Model class with some example values
model = Model(
    num_hidden_layers=3,
    hidden_layer_size=[64, 64, 64],  # Example list for layer sizes
    weight_decay=0.0005,
    learning_rate=0.0001,
    optimizer="Adam",
    activation="sigmoid",
    weight_init="xavier",
    loss = 'cross-entropy'
)


num_classes = int(np.max(y_train)) + 1  # Convert to int
y_train = np.eye(num_classes)[y_train.astype(int)]  # Ensure indices are integers
y_train = np.array([each_y.reshape(-1, 1) for each_y in y_train])

#
model.train(X_train, y_train, 5, 64)

2025-03-15 12:48:52.265351: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-15 12:48:52.270365: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-15 12:48:52.321213: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-15 12:48:52.367729: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742023132.423373    3781 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742023132.43

(60000,)
(60000, 10, 1)
Weights
[(784, 64), (64, 64), (64, 64), (64, 10)]
dw
[(784, 64), (64, 64), (64, 64), (64, 10)]
Epoch number: 1

Epoch: 1, batch: 0/937 completed....Accuracy: 0.078125
Epoch: 1, batch: 100/937 completed....Accuracy: 0.0625
Epoch: 1, batch: 200/937 completed....Accuracy: 0.09375
Epoch: 1, batch: 300/937 completed....Accuracy: 0.125


/home/hiran/Desktop/DL/model.py:187: RuntimeWarning: overflow encountered in divide
  self.v_b[i] = (self.beta2*self.v_b[i] + (1-self.beta2)*db[i]*db[i])/(1 - np.power(self.beta2, self.t) + 1e-8)
/home/hiran/Desktop/DL/model.py:186: RuntimeWarning: overflow encountered in divide
  self.v_w[i] = (self.beta2*self.v_w[i] + (1-self.beta2)*dw[i]*dw[i])/(1 - np.power(self.beta2, self.t) + 1e-8)


Epoch: 1, batch: 400/937 completed....Accuracy: 0.0625
Epoch: 1, batch: 500/937 completed....Accuracy: 0.15625
Epoch: 1, batch: 600/937 completed....Accuracy: 0.046875
Epoch: 1, batch: 700/937 completed....Accuracy: 0.09375
Epoch: 1, batch: 800/937 completed....Accuracy: 0.109375
Epoch: 1, batch: 900/937 completed....Accuracy: 0.140625
Epoch number: 2

Epoch: 2, batch: 0/937 completed....Accuracy: 0.078125
Epoch: 2, batch: 100/937 completed....Accuracy: 0.0625
Epoch: 2, batch: 200/937 completed....Accuracy: 0.09375
Epoch: 2, batch: 300/937 completed....Accuracy: 0.125
Epoch: 2, batch: 400/937 completed....Accuracy: 0.0625
Epoch: 2, batch: 500/937 completed....Accuracy: 0.15625
Epoch: 2, batch: 600/937 completed....Accuracy: 0.046875
Epoch: 2, batch: 700/937 completed....Accuracy: 0.09375
Epoch: 2, batch: 800/937 completed....Accuracy: 0.109375
Epoch: 2, batch: 900/937 completed....Accuracy: 0.140625
Epoch number: 3

Epoch: 3, batch: 0/937 completed....Accuracy: 0.078125
Epoch: 3, batch

In [18]:
X_train.shape
print(np.min(X_train))

0


In [ ]:
import wandb
import json
import itertools
import numpy as np
from keras.datasets import fashion_mnist
from model import Model

# Load configuration from config.json
CONFIG_FILE = "config.json"
with open(CONFIG_FILE, "r") as f:
    config = json.load(f)

# Load dataset
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

# One-hot encode labels
num_classes = int(np.max(y_train)) + 1  
y_train = np.eye(num_classes)[y_train.astype(int)]
y_train = np.array([each_y.reshape(-1, 1) for each_y in y_train])

y_test = np.eye(num_classes)[y_test.astype(int)]
y_test = np.array([each_y.reshape(-1, 1) for each_y in y_test])

# Generate all possible combinations of hyperparameters
param_grid = list(itertools.product(
    config["epochs"],
    config["num_hidden_layers"],
    config["hidden_layer_size"],
    config["weight_decay"],
    config["learning_rate"],
    config["optimizer"],
    config["batch_size"],
    config["weight_initialization"],
    config["activation_functions"]
))

# Iterate over all hyperparameter combinations
for params in param_grid:
    epochs, num_hidden_layers, hidden_layer_size, weight_decay, learning_rate, optimizer, batch_size, weight_init, activation = params

    # Initialize Weights & Biases for each experiment run
    run = wandb.init(
        entity="mm21b030-indian-institute-of-technology-madras",
        project="DL",
        name=f"HL{num_hidden_layers}_HS{hidden_layer_size}_WD{weight_decay}_LR{learning_rate}_OPT{optimizer}_BS{batch_size}_WI{weight_init}_ACT{activation}",
        config={
            "epochs": epochs,
            "num_hidden_layers": num_hidden_layers,
            "hidden_layer_size": hidden_layer_size,
            "weight_decay": weight_decay,
            "learning_rate": learning_rate,
            "optimizer": optimizer,
            "batch_size": batch_size,
            "weight_initialization": weight_init,
            "activation": activation
        }
    )

    # Initialize and train the model
    model = Model(
        num_hidden_layers=num_hidden_layers,
        hidden_layer_size=[hidden_layer_size] * num_hidden_layers,
        weight_decay=weight_decay,
        learning_rate=learning_rate,
        optimizer=optimizer,
        activation=activation,
        weight_init=weight_init,
        loss='cross-entropy'
    )
    
    model.train(X_train, y_train, epochs, batch_size)

    # Evaluate the model
    predictions = model.predict(X_test)
    predicted_classes = np.argmax(predictions, axis=1)  # Ensure correct axis for multi-class classification
    true_classes = np.argmax(y_test, axis=1)  

    accuracy = np.mean(predicted_classes == true_classes)

    # Log accuracy to wandb
    wandb.log({"accuracy": accuracy})

    # Finish the run
    run.finish()


wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mm21b030 (mm21b030-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Weights
[(784, 64), (64, 64), (64, 64), (64, 10)]
dw
[(784, 64), (64, 64), (64, 64), (64, 10)]
Epoch number: 1

Epoch: 1, batch: 0/937 completed....Accuracy: 0.109375


/home/hiran/Desktop/DL/model.py:69: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-x))


Epoch: 1, batch: 100/937 completed....Accuracy: 0.09375
Epoch: 1, batch: 200/937 completed....Accuracy: 0.078125
Epoch: 1, batch: 300/937 completed....Accuracy: 0.078125


/home/hiran/Desktop/DL/model.py:186: RuntimeWarning: overflow encountered in divide
  self.v_w[i] = (self.beta2*self.v_w[i] + (1-self.beta2)*dw[i]*dw[i])/(1 - np.power(self.beta2, self.t) + 1e-8)
/home/hiran/Desktop/DL/model.py:187: RuntimeWarning: overflow encountered in divide
  self.v_b[i] = (self.beta2*self.v_b[i] + (1-self.beta2)*db[i]*db[i])/(1 - np.power(self.beta2, self.t) + 1e-8)


Epoch: 1, batch: 400/937 completed....Accuracy: 0.046875
Epoch: 1, batch: 500/937 completed....Accuracy: 0.140625
Epoch: 1, batch: 600/937 completed....Accuracy: 0.109375
Epoch: 1, batch: 700/937 completed....Accuracy: 0.15625
Epoch: 1, batch: 800/937 completed....Accuracy: 0.09375
Epoch: 1, batch: 900/937 completed....Accuracy: 0.09375


accuracy,▁
accuracy,0


In [1]:
import json
import wandb
# Load hyperparameters from config.json
CONFIG_FILE = "sweep_config.json"

with open(CONFIG_FILE, "r") as f:
    config = json.load(f)

# Define the hyperparameter sweep configuration
sweep_config = {
    "method": "bayes",  # Bayesian optimization for efficient search
    "metric": {"name": "val_accuracy", "goal": "maximize"},
    "parameters": {
        "epochs": {"values": config["epochs"]},
        "num_hidden_layers": {"values": config["num_hidden_layers"]},
        "hidden_layer_size": {"values": config["hidden_layer_size"]},
        "weight_decay": {"values": config["weight_decay"]},
        "learning_rate": {"values": config["learning_rate"]},
        "optimizer": {"values": config["optimizer"]},
        "batch_size": {"values": config["batch_size"]},
        "weight_init": {"values": config["weight_initialization"]},
        "activation": {"values": config["activation_functions"]},
    },
}
with wandb.init(config=config):
        config = wandb.config
        print(config)

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: mm21b030 (mm21b030-indian-institute-of-technology-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


{'epochs': [5, 10], 'num_hidden_layers': [3, 4, 5], 'hidden_layer_size': [32, 64, 128], 'weight_decay': [0, 0.0005, 0.5], 'learning_rate': [0.001, 0.0001], 'optimizer': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam'], 'batch_size': [16, 32, 64], 'weight_initialization': ['random', 'Xavier'], 'activation_functions': ['sigmoid', 'tanh', 'relu']}
